# Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from minisom import MiniSom

# Import post window

In [ ]:
un_post=pd.read_csv('../../WindowedTimeSeries/un_post.csv')
ecd_post=pd.read_csv('../../WindowedTimeSeries/ecd_post.csv')
syn_post=pd.read_csv('../../WindowedTimeSeries/syn_post.csv')

# Loading data

In [ ]:
unsuccessful_post=pd.concat([un_post,ecd_post,syn_post],axis=0)
unsuccessful_post.shape

In [ ]:
ts_list=unsuccessful_post.iloc[:,1:].values.tolist()

# Calling minisom function

In [ ]:
def minisom_func(som_x,som_y,sigma,activation_distance,learning_rate,epochs,neighborhood_function):
    som_x=som_x
    som_y=som_y
    som = MiniSom(som_x,som_y,len(ts_list[0]),sigma=sigma,learning_rate=learning_rate,activation_distance=activation_distance,neighborhood_function=neighborhood_function)
    som.random_weights_init(ts_list)
    print("Training...")
    som.train_random(ts_list,epochs) # random training
    print("\n...ready!")

    return som

In [ ]:
parameters={'som_x':1,'som_y':2,'sigma':0.7,'activation_distance':'euclidean','learning_rate':0.06,'epochs':5000,'neighborhood_function':'bubble'}
som=minisom_func(parameters['som_x'],parameters['som_y'],\
                 parameters['sigma'],parameters['activation_distance'],parameters['learning_rate'],parameters['epochs'],parameters['neighborhood_function'])

In [ ]:
win_map=som.win_map(ts_list)
    
cluster_c = []
cluster_n = []

for x in range(parameters['som_x']):
    for y in range(parameters['som_y']):
        cluster = (x,y)
        if cluster in win_map.keys():
            cluster_c.append(len(win_map[cluster]))
        else:
            cluster_c.append(0)
        cluster_number = x*parameters['som_y']+y+1
        cluster_n.append(f"Cluster {cluster_number}")

plt.figure(figsize=(10,5))
plt.title("Cluster Distribution for SOM")
plt.bar(cluster_n,cluster_c,width=0.5)
plt.show()

cluster_map = []
for idx in range(len(ts_list)):
    winner_node = som.winner(ts_list[idx])
    cluster_number=winner_node[0]*parameters['som_y']+winner_node[1]+1
    cluster_map.append(cluster_number)
unsuccessful_post['cluster']=cluster_map
ecd_cluster=unsuccessful_post.merge(ecd_post,on='TestId')['cluster'].value_counts()/unsuccessful_post.merge(ecd_post,on='TestId')['cluster'].value_counts().sum()
syn_cluster=unsuccessful_post.merge(syn_post,on='TestId')['cluster'].value_counts()/unsuccessful_post.merge(syn_post,on='TestId')['cluster'].value_counts().sum()
un_cluster=unsuccessful_post.merge(un_post,on='TestId')['cluster'].value_counts()/unsuccessful_post.merge(un_post,on='TestId')['cluster'].value_counts().sum()

In [ ]:
print(ecd_cluster,'\n',syn_cluster,'\n',un_cluster,'\n')

## Results:
The results of SOM on post window were bad than results of SOM on cal window